INDUSTRIAL COPPER MODELLING

REGRESSION MODEL

In [51]:
#DecisionTree
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

df= pd.read_csv("copper_preprocessed.csv")

X=df[['quantity tons_log','status','item type','application','thickness_log','width','country','customer','product_ref']]
y=df['selling_price_log']
# encoding categorical variables
ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(X[['item type']])
X_ohe = ohe.fit_transform(X[['item type']]).toarray()
ohe2 = OneHotEncoder(handle_unknown='ignore')
ohe2.fit(X[['status']])
X_be = ohe2.fit_transform(X[['status']]).toarray()
# independent features after encoding
X = np.concatenate((X[['quantity tons_log', 'application', 'thickness_log', 'width','country','customer','product_ref']].values, X_ohe, X_be), axis=1)
scaler = StandardScaler()
X = scaler.fit_transform(X)
# test and train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
# decision tree
dtr = DecisionTreeRegressor()
# hyperparameters
param_grid = {'max_depth': [2, 5, 10, 20],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf': [1, 2, 4],
              'max_features': ['auto', 'sqrt', 'log2']}
# gridsearchcv
grid_search = GridSearchCV(estimator=dtr, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
print("Best hyperparameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
# evalution metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print('Mean squared error:', mse)
print('R-squared:', r2)

c:\Users\SANGAVI\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
180 fits failed out of a total of 540.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\SANGAVI\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\SANGAVI\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "c:\Users\SANGAVI\AppData\Local\Programs\Python\Python312\Lib\site-packages\skle

Best hyperparameters: {'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 10}
Mean squared error: 0.027367974782014855
R-squared: 0.916815090264562


In [52]:
# ['quantity tons_log', 'application', 'thickness_log', 'width','country','customer','product_ref']].values, X_ohe, X_be
new_sample = np.array([[np.log(40), 10, np.log(250), 0, 28,30202938,1670798778,'PL','Won']])
new_sample_ohe = ohe.transform(new_sample[:, [7]]).toarray()
new_sample_be = ohe2.transform(new_sample[:, [8]]).toarray()
new_sample = np.concatenate((new_sample[:, [0,1,2, 3, 4, 5, 6,]], new_sample_ohe, new_sample_be), axis=1)
new_sample1 = scaler.transform(new_sample)
new_pred = best_model.predict(new_sample1)
print('Predicted selling price:', np.exp(new_pred))

Predicted selling price: [1103.73085412]


c:\Users\SANGAVI\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\SANGAVI\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [14]:
import pandas as pd
df = pd.read_csv("copper_preprocessed.csv")

In [15]:
df.columns

Index(['Unnamed: 0', 'id', 'item_date', 'quantity tons', 'customer', 'country',
       'status', 'item type', 'application', 'thickness', 'width',
       'material_ref', 'product_ref', 'delivery date', 'selling_price',
       'selling_price_log', 'quantity tons_log', 'thickness_log'],
      dtype='object')

In [57]:
#Randomforest Final
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score

df = pd.read_csv("copper_preprocessed.csv")

X = df[
    [
        "quantity tons_log",
        "status",
        "item type",
        "application",
        "thickness_log",
        "width",
        "country",
        "customer",
        "product_ref",
    ]
]
y = df["selling_price_log"]

# Encode categorical features
ohe = OneHotEncoder(handle_unknown="ignore")
ohe.fit(X[["item type", "status"]])  # Encode both 'item type' and 'status'
X_ohe = ohe.transform(X[["item type", "status"]]).toarray()

# Combine numerical and encoded features
X = np.concatenate(
    (
        X[
            [
                "quantity tons_log",
                "application",
                "thickness_log",
                "width",
                "country",
                "customer",
                "product_ref",
            ]
        ].values,
        X_ohe,
    ),
    axis=1,
)

scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Random Forest Regressor (using default hyperparameters)
rfr = RandomForestRegressor()

# Train the model
rfr.fit(X_train, y_train)

# Make predictions on test set
y_pred = rfr.predict(X_test)

# Evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean squared error:", mse)
print("R-squared:", r2)

# Save model and scaler for future predictions
with open("random_forest_model.pkl", "wb") as f:
    pickle.dump(rfr, f)
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)
with open("ohe.pkl", "wb") as f:
    pickle.dump(ohe, f)

print("Model, scaler, and encoder saved for future predictions!")


Mean squared error: 0.019899021323904264
R-squared: 0.9395169607602705
Model, scaler, and encoder saved for future predictions!


In [56]:
#ExtraTreesRegressor 2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score

df = pd.read_csv("copper_preprocessed.csv")

X = df[
    [
        "quantity tons_log",
        "status",
        "item type",
        "application",
        "thickness_log",
        "width",
        "country",
        "customer",
        "product_ref",
    ]
]
y = df["selling_price_log"]

# Encode categorical features
ohe = OneHotEncoder(handle_unknown="ignore")
ohe.fit(X[["item type", "status"]])  # Encode both 'item type' and 'status'
X_ohe = ohe.transform(X[["item type", "status"]]).toarray()

# Combine numerical and encoded features
X = np.concatenate(
    (
        X[
            [
                "quantity tons_log",
                "application",
                "thickness_log",
                "width",
                "country",
                "customer",
                "product_ref",
            ]
        ].values,
        X_ohe,
    ),
    axis=1,
)

scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# ExtraTreesRegressor (using default hyperparameters)
etr = ExtraTreesRegressor()

# Train the model
etr.fit(X_train, y_train)

# Make predictions on test set
y_pred = etr.predict(X_test)

# Evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean squared error:", mse)
print("R-squared:", r2)


Mean squared error: 0.024104700756109317
R-squared: 0.9267338057504203


CLASSIFICATION

In [3]:
pip install xgboost

   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB 435.7 kB/s eta 0:03:49
   ---------------------------------------- 0.1/99.8 MB 465.5 kB/s eta 0:03:35
   ---------------------------------------- 0.1/99.8 MB 726.2 kB/s eta 0:02:18
   ---------------------------------------- 0.2/99.8 MB 1.2 MB/s eta 0:01:27
   ---------------------------------------- 0.5/99.8 MB 1.8 MB/s eta 0:00:54
   ---------------------------------------- 0.6/99.8 MB 2.0 MB/s eta 0:00:49
   ---------------------------------------- 1.0/99.8 MB 3.2 MB/s eta 0:00:32
    --------------------------------------- 1.3/99.8 MB 3.4 MB/s eta 0:00:30
    --------------------------------------- 2.2/99.8 MB 5.2 MB/s eta 0:00:19
    --------------------------------------- 2.2/99.8 MB 5.2 MB/s eta 0:00:19
   - ---------

In [37]:
#Comparing ExtraTreesClassifier, XGBClassifier,LogisticRegression
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,confusion_matrix

# Load your data from a CSV file (replace 'your_data.csv' with your actual file path)
df_cleaned = pd.read_csv("copper_preprocessed.csv")
data = df_cleaned[df_cleaned['status'].isin(['Won', 'Lost'])]


# Select features for prediction (excluding unnecessary ones like 'id' and 'item_date')
features = ['quantity tons_log','selling_price_log','application','thickness_log','width','country','customer','product_ref']

# Separate target variable (status)
target = 'status'
label_encoder = LabelEncoder()
data['status'] = label_encoder.fit_transform(data['status'])

# Handle missing values (replace with your preferred method)
# ... (e.g., using imputation techniques)

# Encode categorical features
categorical_features = ['item type']
label_encoder = LabelEncoder()
encoded_data = pd.DataFrame()
for feature in categorical_features:
    data[feature] = label_encoder.fit_transform(data[feature])
    encoded_data[feature] = data[feature]

# Combine numerical and encoded categorical features
data_processed = pd.concat([data[features], encoded_data], axis=1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_processed, data[target], test_size=0.2, random_state=42)

# Define and train the ExtraTreesClassifier model
model_extra_trees = ExtraTreesClassifier(n_estimators=100, random_state=42)
model_extra_trees.fit(X_train, y_train)

# Make predictions on the testing set using ExtraTreesClassifier
y_predicted_extra_trees = model_extra_trees.predict(X_test)

# Evaluate ExtraTreesClassifier accuracy
accuracy_extra_trees = accuracy_score(y_test, y_predicted_extra_trees)
precision_extra_trees = precision_score(y_test, y_predicted_extra_trees)
recall_extra_trees = recall_score(y_test, y_predicted_extra_trees)  
f1_extra_trees = f1_score(y_test, y_predicted_extra_trees)  
cm1 = confusion_matrix(y_test, y_predicted_extra_trees)
print("ExtraTreesClassifier Accuracy:", accuracy_extra_trees)
print("ExtraTreesClassifier Precision:", precision_extra_trees)
print("ExtraTreesClassifier Recall:", recall_extra_trees)
print("ExtraTreesClassifier F1-score:", f1_extra_trees)
print(f"Confusion Matrix:\n{cm1}")

# Define and train the XGBoost Classifier model
model_xgboost = XGBClassifier(objective='binary:logistic', n_estimators=100, random_state=42)
model_xgboost.fit(X_train, y_train)

# Make predictions on the testing set using XGBoostClassifier
y_predicted_xgboost = model_xgboost.predict(X_test)

# Evaluate XGBoostClassifier accuracy
accuracy_xgboost = accuracy_score(y_test, y_predicted_xgboost)
precision_xgboost = precision_score(y_test, y_predicted_xgboost)
recall_xgboost = recall_score(y_test, y_predicted_xgboost)
f1_xgboost = f1_score(y_test, y_predicted_xgboost)
cm2 = confusion_matrix(y_test, y_predicted_xgboost)
print("XGBoostClassifier Accuracy:", accuracy_xgboost)
print("XGBoostClassifier Precision:", precision_xgboost)
print("XGBoostClassifier Recall:", recall_xgboost)
print("XGBoostClassifier F1-score:", f1_xgboost)
print(f"Confusion Matrix:\n{cm2}")

model_logistic = LogisticRegression(random_state=42)  # Set random state for reproducibility
model_logistic.fit(X_train, y_train)

# Make predictions on the testing set using Logistic Regression
y_predicted_logistic = model_logistic.predict(X_test)

# Evaluate Logistic Regression accuracy
accuracy_logistic = accuracy_score(y_test, y_predicted_logistic)
precision_logistic = precision_score(y_test, y_predicted_logistic)
recall_logistic = recall_score(y_test, y_predicted_logistic)
f1_logistic = f1_score(y_test, y_predicted_logistic)
cm3 = confusion_matrix(y_test, y_predicted_logistic)
print("Logistic Regression Accuracy:", accuracy_logistic)
print("Logistic Regression Precision:", precision_logistic)
print("Logistic Regression Recall:", recall_logistic)
print("Logistic Regression F1-score:", f1_logistic)
print(f"Confusion Matrix:\n{cm3}")

# You can choose the model with the highest accuracy for further analysis or exploration.


C:\Users\SANGAVI\AppData\Local\Temp\ipykernel_21336\305719523.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['status'] = label_encoder.fit_transform(data['status'])
C:\Users\SANGAVI\AppData\Local\Temp\ipykernel_21336\305719523.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[feature] = label_encoder.fit_transform(data[feature])


ExtraTreesClassifier Accuracy: 0.9351611831173148
ExtraTreesClassifier Precision: 0.9533461620697028
ExtraTreesClassifier Recall: 0.962857389651896
ExtraTreesClassifier F1-score: 0.9580781709963686
Confusion Matrix:
[[ 5845  1091]
 [  860 22294]]
XGBoostClassifier Accuracy: 0.8993020937188435
XGBoostClassifier Precision: 0.9163356504468719
XGBoostClassifier Recall: 0.9564654055454781
XGBoostClassifier F1-score: 0.9359705845061493
Confusion Matrix:
[[ 4914  2022]
 [ 1008 22146]]
Logistic Regression Accuracy: 0.7694915254237288
Logistic Regression Precision: 0.7694915254237288
Logistic Regression Recall: 1.0
Logistic Regression F1-score: 0.8697318007662835
Confusion Matrix:
[[    0  6936]
 [    0 23154]]


In [10]:
#XGBClassifier with gridcv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from xgboost import XGBClassifier
from sklearn.utils.class_weight  import compute_class_weight
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load your data from a CSV file (replace 'your_data.csv' with your actual file path)
df_cleaned = pd.read_csv("copper_preprocessed.csv")
data = df_cleaned[df_cleaned['status'].isin(['Won', 'Lost'])]

# Select features for prediction (excluding unnecessary ones like 'id' and 'item_date')
features = ['quantity tons','thickness', 'width']

# Separate target variable (status)
target = 'status'
label_encoder = LabelEncoder()
data['status'] = label_encoder.fit_transform(data['status'])

# Handle missing values (replace with your preferred method)
# ... (e.g., using imputation techniques)

# Encode categorical features
categorical_features = ['customer', 'country', 'item type', 'application', 'material_ref', 'product_ref']
label_encoder = LabelEncoder()
encoded_data = pd.DataFrame()
for feature in categorical_features:
    data[feature] = label_encoder.fit_transform(data[feature])
    encoded_data[feature] = data[feature]

# Combine numerical and encoded categorical features
data_processed = pd.concat([data[features], encoded_data], axis=1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_processed, data[target], test_size=0.2, random_state=42)

# Calculate class weights
class_weight = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight = dict(zip(np.unique(y_train), class_weight))

param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 8],
    'gamma': [0, 1, 3],
    'colsample_bytree': [0.5, 0.7, 1]
}

# Define and train the XGBoost Classifier model
model_xgboost = XGBClassifier(objective='binary:logistic', n_estimators=100, random_state=42, scale_pos_weight=class_weight[1])
grid_search = GridSearchCV(estimator=model_xgboost, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best model and parameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
y_predicted_xgboost = best_model.predict(X_test)

# Make predictions on the testing set using XGBoostClassifier
# y_predicted_xgboost = model_xgboost.predict(X_test)

# Evaluate XGBoostClassifier accuracy
accuracy_xgboost = accuracy_score(y_test, y_predicted_xgboost)
precision_xgboost = precision_score(y_test, y_predicted_xgboost)
recall_xgboost = recall_score(y_test, y_predicted_xgboost)
f1_xgboost = f1_score(y_test, y_predicted_xgboost)
print("XGBoostClassifier Accuracy:", accuracy_xgboost)
print("XGBoostClassifier Precision:", precision_xgboost)
print("XGBoostClassifier Recall:", recall_xgboost)
print("XGBoostClassifier F1-score:", f1_xgboost)


C:\Users\SANGAVI\AppData\Local\Temp\ipykernel_21336\596411381.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['status'] = label_encoder.fit_transform(data['status'])
C:\Users\SANGAVI\AppData\Local\Temp\ipykernel_21336\596411381.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[feature] = label_encoder.fit_transform(data[feature])
C:\Users\SANGAVI\AppData\Local\Temp\ipykernel_21336\596411381.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

XGBoostClassifier Accuracy: 0.8950481887670322
XGBoostClassifier Precision: 0.9363652234636871
XGBoostClassifier Recall: 0.9265785609397944
XGBoostClassifier F1-score: 0.9314461859071766


In [27]:
#Final ExtraTreesClassifier for status prediction 
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,confusion_matrix

# Load your data from a CSV file (replace 'your_data.csv' with your actual file path)
df_cleaned = pd.read_csv("copper_preprocessed.csv")
data = df_cleaned[df_cleaned['status'].isin(['Won', 'Lost'])]

# Select features for prediction (excluding unnecessary ones like 'id' and 'item_date')
features = ['quantity tons_log','selling_price_log','application','thickness_log','width','country','customer','product_ref']

# Separate target variable (status)
target = 'status'
label_encoder_target = LabelEncoder()  # Create a separate LabelEncoder for the target
data['status'] = label_encoder_target.fit_transform(data['status'])

# Handle missing values (replace with your preferred method)
# ... (e.g., using imputation techniques)

# Encode categorical features
categorical_features = ['item type']
label_encoders = {}  # Create a dictionary to store LabelEncoders for categorical features
for feature in categorical_features:
  label_encoders[feature] = LabelEncoder()  # Create a LabelEncoder for each feature
  data[feature] = label_encoders[feature].fit_transform(data[feature])

# Combine numerical and encoded categorical features
data_processed = pd.concat([data[features], data[categorical_features]], axis=1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_processed, data[target], test_size=0.2, random_state=42)

# Calculate class weights
class_weight = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight = dict(zip(np.unique(y_train), class_weight))

# Define and train the ExtraTreesClassifier model
model_et = ExtraTreesClassifier(n_estimators=100, random_state=42, class_weight=class_weight)
model_et.fit(X_train, y_train)

# Make predictions on the testing set using ExtraTreesClassifier
y_predicted_extra_trees = model_et.predict(X_test)

# Evaluate ExtraTreesClassifier accuracy
accuracy_extra_trees = accuracy_score(y_test, y_predicted_extra_trees)
precision_extra_trees = precision_score(y_test, y_predicted_extra_trees)
recall_extra_trees = recall_score(y_test, y_predicted_extra_trees)
f1_extra_trees = f1_score(y_test, y_predicted_extra_trees)
cm = confusion_matrix(y_test, y_predicted_extra_trees)

print("ExtraTreesClassifier Accuracy:", accuracy_extra_trees)
print("ExtraTreesClassifier Precision:", precision_extra_trees)
print("ExtraTreesClassifier Recall:", recall_extra_trees)
print("ExtraTreesClassifier F1-score:", f1_extra_trees)
print(f"Confusion Matrix:\n{cm}")

# Pickle the model and label encoders (replace filenames if needed)
with open('label_encoder_target.pkl', 'wb') as pickle_file:
  pickle.dump(label_encoder_target, pickle_file)

print("Label Encoder for Target Pickled Successfully!")
with open('extra_trees_model.pkl', 'wb') as pickle_file:
  pickle.dump(model_et, pickle_file)

with open('label_encoders.pkl', 'wb') as pickle_file:
  pickle.dump(label_encoders, pickle_file)

print("ExtraTreesClassifier Model and LabelEncoders Pickled Successfully!")


C:\Users\SANGAVI\AppData\Local\Temp\ipykernel_23676\3662422753.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['status'] = label_encoder_target.fit_transform(data['status'])
C:\Users\SANGAVI\AppData\Local\Temp\ipykernel_23676\3662422753.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[feature] = label_encoders[feature].fit_transform(data[feature])


ExtraTreesClassifier Accuracy: 0.9347623795280824
ExtraTreesClassifier Precision: 0.9527024140140995
ExtraTreesClassifier Recall: 0.9630301459790965
ExtraTreesClassifier F1-score: 0.957838441547284
Confusion Matrix:
[[ 5829  1107]
 [  856 22298]]
Label Encoder for Target Pickled Successfully!
ExtraTreesClassifier Model and LabelEncoders Pickled Successfully!


In [20]:
#Function to predict selling price using random forest model
import pandas as pd
import numpy as np
import pickle
from math import exp 


def predict_selling_price(data, model_file="random_forest_model.pkl", scaler_file="scaler.pkl", ohe_file="ohe.pkl"):
  """
  Predicts selling price using a trained Random Forest model and saved scaler and encoder.

  Args:
      data (pandas.DataFrame): DataFrame containing features for prediction.
      model_file (str, optional): Path to the pickled Random Forest model file. Defaults to "random_forest_model.pkl".
      scaler_file (str, optional): Path to the pickled scaler file. Defaults to "scaler.pkl".
      ohe_file (str, optional): Path to the pickled OneHotEncoder file. Defaults to "ohe.pkl".

  Returns:
      float: Predicted selling price.
  """

  # Load the model, scaler, and encoder
  with open(model_file, "rb") as f:
      model = pickle.load(f)
  with open(scaler_file, "rb") as f:
      scaler = pickle.load(f)
  with open(ohe_file, "rb") as f:
      ohe = pickle.load(f)

  # Extract features from data
  selling_price_features = [
      "quantity tons_log",
      "application",
      "thickness_log",
      "width",
      "country",
      "customer",
      "product_ref",
  ]
  X = data[selling_price_features]

  # Encode categorical features
  X_ohe = ohe.transform(data[["item type", "status"]]).toarray()

  # Combine numerical and encoded features
  X = np.concatenate(
      (
          X[[
              "quantity tons_log",
              "application",
              "thickness_log",
              "width",
              "country",
              "customer",
              "product_ref",
          ]].values,
          X_ohe,
      ),
      axis=1,
  )

  # Scale the features
  X = scaler.transform(X)

  # Make prediction
  predicted_price = model.predict(X)[0]  # Assuming it returns an array, return the first element

   # Convert predicted price from log scale to actual price
  actual_selling_price = exp(predicted_price)

  return actual_selling_price

# Example usage
data = pd.DataFrame({
  "quantity tons_log": [55],
  "application": [10],
  "thickness_log": [2],
  "width": [1200],
  "item type": ['W'],
  "status": ['Won'],
  "country": [28],
  "customer": [30156308.00],
  "product_ref": [1670798778],
})

predicted_price = predict_selling_price(data)
print("Predicted Selling Price:", predicted_price)


Predicted Selling Price: 623.6323023876766


In [37]:
#Function to predict the status using ExtraTreesClassifier
import pandas as pd
import pickle


def predict_status(data, model_file="extra_trees_model.pkl", label_encoder_target_file="label_encoder_target.pkl", label_encoders_file="label_encoders.pkl"):
  """
  Predicts the status ("Won" or "Lost") for a new data point using a trained ExtraTreesClassifier model and saved label encoders.

  Args:
      data (pandas.DataFrame): DataFrame containing features for prediction.
      model_file (str, optional): Path to the pickled ExtraTreesClassifier model file. Defaults to "extra_trees_model.pkl".
      label_encoder_target_file (str, optional): Path to the pickled LabelEncoder file for the target variable. Defaults to "label_encoder_target.pkl".
      label_encoders_file (str, optional): Path to the pickled dictionary containing LabelEncoders for categorical features. Defaults to "label_encoders.pkl".

  Returns:
      str: Predicted status ("Won" or "Lost").
  """

  # Load the model and label encoders
  with open(model_file, "rb") as f:
      model = pickle.load(f)
  with open(label_encoder_target_file, "rb") as f:
      label_encoder_target = pickle.load(f)
  with open(label_encoders_file, "rb") as f:
      label_encoders = pickle.load(f)

  # Extract features from data
  features = ['quantity tons_log', 'selling_price_log',  'application',
 'thickness_log', 'width', 'country', 'customer', 'product_ref','item type']
  data_processed = data[features]

  # Encode categorical features
  categorical_features = ['item type']
  for feature in categorical_features:
      data_processed[feature] = label_encoders[feature].transform(data_processed[feature])

  # Make prediction
  predict_status = model.predict(data_processed)
#   print(predict_status.shape)
  predicted_status_decoded = label_encoder_target.inverse_transform(predict_status)


  return predicted_status_decoded

# Example usage
data = pd.DataFrame({
  "quantity tons_log": [55],
  "selling_price_log": [3.5],
  "application": [10],
  "thickness_log": [0.8],
  "width": [1200],
  "country": [28],
  "customer": [30156308.00],
  "product_ref": [1670798778],
  "item type": ['IPL']
})

predicted_status = predict_status(data)
print("Predicted Status:", predicted_status[0])


Predicted Status: Won
